In [ ]:
from __future__ import division, print_function, absolute_import 

import matplotlib.pyplot as plt
%matplotlib inline
# import matplotlib.ticker
import matplotlib.cm as cm
import matplotlib.colors as colors
from matplotlib.font_manager import FontProperties
import numpy as np
import astropy.io.ascii as at
import astropy.io.fits as fits
import astropy.table as table
import astropy.units as u
from astropy.coordinates import SkyCoord

# A few definitions

In [ ]:
single_figure = (8,8)

In [ ]:
import matplotlib as mpl
mpl.rcParams['lines.markeredgewidth'] = 1.5
mpl.rcParams['lines.markersize'] = 8
mpl.rcParams['font.size'] = 16
mpl.rcParams['axes.titlesize'] = 24
mpl.rcParams['axes.labelsize'] = 22
mpl.rcParams['xtick.major.size'] = 6
mpl.rcParams['xtick.minor.size'] = 3
mpl.rcParams['xtick.major.width'] = 1.5
mpl.rcParams['xtick.minor.width'] = 1
mpl.rcParams['xtick.labelsize'] = 18
mpl.rcParams['ytick.major.size'] = 6
mpl.rcParams['ytick.minor.size'] = 3
mpl.rcParams['ytick.major.width'] = 1.5
mpl.rcParams['ytick.minor.width'] = 1
mpl.rcParams['ytick.labelsize'] = 18
mpl.rcParams['legend.frameon'] = False
mpl.rcParams['legend.numpoints'] = 1
mpl.rcParams['legend.fontsize'] = 16
# mpl.rcParams[''] = 

# Astropy angles and SkyCoord

from http://astropy-tutorials.readthedocs.io/en/latest/rst-tutorials/Coordinates-Intro.html

The `SkyCoord` class in the `astropy.coordinates` package is used to represent celestial coordinates. First, we'll make a SkyCoord object based on our object's name, "Praesepe", or "M44" for short. Most astronomical object names can be found by [SESAME](http://cdsweb.u-strasbg.fr/cgi-bin/Sesame), a service which queries Simbad, NED, and VizieR and returns the object's type and its J2000 position. This service can be used via the `SkyCoord.from_name()` [class method](https://julien.danjou.info/blog/2013/guide-python-static-class-abstract-methods):

In [ ]:
# initialize a SkyCood object named m44_center at the location of Praesepe
m44_center = SkyCoord.from_name('Praesepe')

In [ ]:
# uncomment and run these lines if you don't have an internet connection
# m44_center = SkyCoord(130.1*u.degree, 19.6667*u.degree, frame='icrs')

Show the available methods and attributes of the SkyCoord object we've created called `m44_center`

In [ ]:
type(m44_center)

In [ ]:
dir(m44_center)

Show the RA and Dec.

In [ ]:
print(m44_center.ra, m44_center.dec)
print(m44_center.ra.hour, m44_center.dec)

This object we just created has various useful ways of accessing the information contained within it.  In particular, the ``ra`` and ``dec`` attributes are specialized [``Quantity``](http://docs.astropy.org/en/stable/units/index.html) objects (actually, a subclass called [``Angle``](http://docs.astropy.org/en/stable/api/astropy.coordinates.Angle.html), which in turn is subclassed by [``Latitude``](http://docs.astropy.org/en/stable/api/astropy.coordinates.Latitude.html) and [``Longitude``](http://docs.astropy.org/en/stable/api/astropy.coordinates.Longitude.html)).  These objects store angles and provide pretty representations of those angles, as well as some useful attributes to quickly convert to common angle units:

In [ ]:
type(m44_center.ra), type(m44_center.dec)

In [ ]:
m44_center.ra, m44_center.dec

In [ ]:
m44_center

In [ ]:
m44_center.ra.hour

SkyCoord will also accept string-formatted coordinates either as separate strings for ra/dec or a single string.  You'll have to give units, though, if they aren't part of the string itself.

In [ ]:
SkyCoord("08:40:24 +19:59:00", unit=(u.hourangle,u.degree), frame='icrs')

# Plot the distribution of open clusters on the sky

## First read in the cluster data

Kharchenko et al. (2013) looked for clusters primarily in the Galactic Plane 

In [ ]:
# Read in the cluster catalog
khar_full = at.read("kharchenko2013_clusters.csv",delimiter=",",
                data_start=1,header_start=0)
# M44 and the Hyades aren't in there (too nearby to appear as an overdensity)
# Print the catalog columns
print(khar_full.dtype)

In [ ]:
# We're only interested in Open Clusters, which have a "Type" set in the catalog
ocs = khar_full["Type"].mask.nonzero()[0]
print(ocs)
# Select a smaller catalog, made only of OCs
khar = khar_full[ocs]

In [ ]:
# Compute the height of each cluster above the Galactic Plane
# GLAT is the cluster's galactic latitude, and "d" is its distance
# Then we just use simple Trig
khar_z = np.sin(khar["GLAT"] * np.pi / 180) * khar["d"]
# print(khar_z)


# Define a SkyCoord object using the RA (longitude on the sky)
# and Decliation (latitude on the sky) of each cluster
# In this case, we're passing in arrays of RA and Dec, 
# and then defining the unit separately
khar_pos = SkyCoord(khar["RAJ2000"],khar["DEJ2000"],
                     unit=u.deg,frame="icrs")

Schmeja et al. (2014) extended the Kharchenko catalog, but focused on far away clusters that were missed in their initial search

In [ ]:
# Read in the catalog
schm = at.read("schmeja2014_clusters.csv",delimiter=",",
               data_start=1,header_start=0)
print(schm.dtype)

# Also compute the height above the Galactic plane
# and generate a SkyCoord object for the Schmeja clusters
schm_z = np.sin(schm["GLAT"] * np.pi / 180) * schm["d"]
schm_pos = SkyCoord(schm["RAJ2000"],schm["DEJ2000"],unit=u.deg,frame="icrs")

## Now plot the clusters

First, before we get to different projections, let's just plot the clusters in RA and Dec

In [ ]:
plt.figure(figsize=(10,8))
# Define the subplot axis
ax = plt.subplot(111)

# Now plot the RA and Dec of each cluster
ax.plot(khar_pos.ra,khar_pos.dec,
        'kx',alpha=0.5,label="Kharchenko+2013")
ax.plot(schm_pos.ra,schm_pos.dec,
        'ks',mfc="none",alpha=0.5,label="Schmeja+2014")

# Now we re-label the axes to be the familiar hours of RA
# If we were working with Earth-based data, then you could skip this step
# and leave longitude as degrees from -180 to 180
# ax.set_xticklabels(['14h','16h','18h','20h','22h',
#                     '0h','2h','4h','6h','8h','10h'])

# Turn on the axes background grid
ax.grid(True)
ax.set_xlabel("RA")
ax.set_ylabel("Dec")
ax.set_xlim(0,360)

_ = plt.title("Equatorial Coords")

For later, though, we're going to want to center RA=0hr, or 180 degrees. We have to force the RA to "wrap", so that it runs from -180 to 180 instead of 0 to 360, and then relabel the axes

In [ ]:
plt.figure(figsize=(10,8))
# Define the subplot axis
ax = plt.subplot(111)

# Now plot the RA and Dec of each cluster
ax.plot(khar_pos.ra.wrap_at(180 * u.deg),khar_pos.dec,
        'kx',alpha=0.5,label="Kharchenko+2013")
ax.plot(schm_pos.ra.wrap_at(180 * u.deg),schm_pos.dec,
        'ks',mfc="none",alpha=0.5,label="Schmeja+2014")

ax.set_xlim(-180,180)
ax.set_xticks(np.arange(-180,181,30))
# Because the convention is to have RA run from 0-360,
# we have to re-label the axes. 
# For Earth-based data you don't need to do this
ax.set_xticklabels(np.append(np.arange(180,360,30),np.arange(0,181,30)))

ax.set_xlabel("RA")
ax.set_ylabel("Dec")

# Turn on the axes background grid
ax.grid(True)

_ = plt.title("Equatorial Coords")

# What if we want to plot the clusters in a sky projection?

For more information on using geometric projections in Matplotlib, see 
https://matplotlib.org/gallery/subplots_axes_and_figures/geo_demo.html

First, we'll plot in the "aitoff" projection - this tries to compromise between minimizing distortion while representing distances accurately

Note that now we have to convert the coordinates to radians before plotting - thankfully, `SkyCoord` makes this easy! Just use ``.radian``

In [ ]:
plt.figure(figsize=(10,8))
# Define the subplot axis, and set the appropriate projection keyword
ax = plt.subplot(111, projection="aitoff")

# Plot the RA and Dec of each cluster
# Now, not only do we have to force the 
ax.plot(khar_pos.ra.wrap_at(180 * u.deg).radian,
        khar_pos.dec.radian,
        'kx',alpha=0.5,label="Kharchenko+2013")
ax.plot(schm_pos.ra.wrap_at(180 * u.deg).radian,
        schm_pos.dec.radian,
        'ks',mfc="none",alpha=0.5,label="Schmeja+2014")

# Re-label the axes
ax.set_xticklabels(np.append(np.arange(210,360,30),np.arange(0,181,30)))

# Turn on the axes background grid
ax.grid(True)

_ = plt.title("Ecliptic Coords, Aitoff Projection")

So the above plot is in Equatorial coordinates - as though you projected the Earth's lat/long outward onto the sky in RA/Dec. The plane of the solar system is shown in the line, and the Galactic plane is shown by the location of all those clusters

We can plot this in some other fun projections as well

In [ ]:
plt.figure(figsize=(10,8))
# Define the subplot axis, and set the appropriate projection keyword
ax = plt.subplot(111, projection="mollweide")

# Plot the RA and Dec of each cluster
# Now, not only do we have to force the 
ax.plot(khar_pos.ra.wrap_at(180 * u.deg).radian,
        khar_pos.dec.radian,
        'kx',alpha=0.5,label="Kharchenko+2013")
ax.plot(schm_pos.ra.wrap_at(180 * u.deg).radian,
        schm_pos.dec.radian,
        'ks',mfc="none",alpha=0.5,label="Schmeja+2014")

# Re-label the axes
ax.set_xticklabels(np.append(np.arange(210,360,30),np.arange(0,181,30)))

# Turn on the axes background grid
ax.grid(True)
_ = plt.title("Ecliptic Coords, Mollweide Projection")

In [ ]:
plt.figure(figsize=(10,8))
# Define the subplot axis, and set the appropriate projection keyword
ax = plt.subplot(111, projection="lambert")

# Plot the RA and Dec of each cluster
# Now, not only do we have to force the 
ax.plot(khar_pos.ra.wrap_at(180 * u.deg).radian,
        khar_pos.dec.radian,
        'kx',alpha=0.5,label="Kharchenko+2013")
ax.plot(schm_pos.ra.wrap_at(180 * u.deg).radian,
        schm_pos.dec.radian,
        'ks',mfc="none",alpha=0.5,label="Schmeja+2014")

# Re-label the axes
ax.set_xticklabels(np.append(np.arange(210,360,30),np.arange(0,181,30)))

# Turn on the axes background grid
ax.grid(True)
_ = plt.title("Ecliptic Coords, Lambert Projection")

## Galactic Coordinates

In [ ]:
plt.figure(figsize=single_figure)
ax1 = plt.subplot(211)

ax1.plot(khar["GLON"],khar_z,'kx',alpha=0.5)
ax1.plot(schm["GLON"],schm_z,'ks',mfc="none",alpha=0.5,label="Schmeja+2014")

# ax1.set_xscale("log")
ax1.set_xlim(0,360)
ax1.set_ylim(-4e3,4e3)

# ax1.set_xlabel("Distance (pc)")
ax1.set_ylabel("Z (pc)")


ax2 = plt.subplot(212,sharex=ax1)

ax2.plot(khar["GLON"],khar["GLAT"],'kx',alpha=0.5)
ax2.plot(schm["GLON"],schm["GLAT"],'ks',mfc="none",alpha=0.5,label="Schmeja+2014")

# ax2.set_xscale("log")
# ax.set_yscale("log")

ax2.set_ylim(-90,90)
ax2.set_xlim(0,360)

ax2.set_xlabel("l (deg)")
ax2.set_ylabel("b (deg)")



### Transform to Galactic Coordinates

Now let's go back to the aitoff projection, but plotting in Galactic coordinates, instead of Ecliptic coordinages

In [ ]:
plt.figure(figsize=(10,8))
ax = plt.subplot(111, projection="aitoff")

# Astropy SkyCoord positions automatically include Galactic coordinates
# accessible using .galactic.l (for Galactic longitude)
# and .galactic.b (for Galactic latitude)
# And again we have to wrap the longitude at 180 degrees
# in order to center 0 on our plot
ax.plot(khar_pos.galactic.l.wrap_at(180 * u.deg).radian,
         khar_pos.galactic.b.radian,
         'kx',alpha=0.5,label="Kharchenko+2013")
ax.plot(schm_pos.galactic.l.wrap_at(180 * u.deg).radian,schm_pos.galactic.b.radian,
         'ks',mfc="none",alpha=0.5,label="Schmeja+2014")

ax.set_xticklabels([r"210$\degree$",r"240$\degree$",r"270$\degree$",
                   r"300$\degree$",r"330$\degree$",r"0$\degree$",
                   r"30$\degree$",r"60$\degree$",r"90$\degree$",
                   r"120$\degree$",r"150$\degree$",])
# ax.set_yticklabels([])
ax.grid(True)

# We have to rearrange the Ecliptic coordinates here too
ecliptic_l = ecliptic.galactic.l.wrap_at(180 * u.deg).radian
ecliptic_b = ecliptic.galactic.b.radian
new_ecl = np.argsort(ecliptic_l)
ecliptic_l = ecliptic_l[new_ecl]
ecliptic_b = ecliptic_b[new_ecl]

ax.plot(ecliptic_l,ecliptic_b,'-',lw=2,zorder=-1)

_ = plt.title("Galactic Coords, Aitoff Projection")